In [4]:
import numpy as np
import matplotlib
import json
import pandas
import os
import spacy
from collections import defaultdict

nlp = spacy.load('en_core_web_sm')

In [5]:
def tokenize(text):
    doc = nlp(text)
    return [tok.text for tok in doc]


In [ ]:
splits = ["train", "valid_seen", "valid_unseen", "tests_seen", "tests_unseen"]
task_fields = ["task_type", "focus_object", "base_object", "dest_object", "scene"]
stats_dict = defaultdict(lambda: [])

for split in splits:
    for task_dir in os.listdir(split):
        task_values = task_dir.split("-")
        for trial_dir in os.listdir("{}/{}".format(split, task_dir)):
            stats_dict["task_id"].append(trial_dir)
            stats_dict["split"].append(split)
            for i, field in enumerate(task_fields):
                stats_dict[field].append(task_values[i])
            traj_data = json.load("{}/{}/traj_data.json".format(split, trial_dir))
            num_steps_list = []
            num_step_tokens_list = []
            num_task_tokens_list = []
            for directive in traj_data["turk_annotations"]["anns"]:
                num_steps_list.append(len(directive["high_descs"]))
                num_step_tokens_list.append(sum([len(tokenize(desc)) for desc in directive["high_descs"]]))
                num_task_tokens_list.append(len(tokenize(directive["task_desc"])))
            stats_dict["num_steps"].append(np.mean(num_steps_list))
            stats_dict["steps_num_toks"].append(np.mean(num_step_tokens_list))
            stats_dict["task_num_toks"].append(np.mean(num_task_tokens_list))
            stats_dict["timesteps"].append(len(traj_data["plan"]["low_actions"]))
            navigation = 0
            interaction = 0
            for action in traj_data["plan"]["low_actions"]:
                if "mask" in action["args"].keys():
                    interaction += 1
                else:
                    navigation += 1
        





  